# ALMA-IMF+SPICY: SED Fitting

Workspace for importing the fits files generated through the SED Table Prep workspace, running the SED fitting, and saving the results to an astropy table.

In [1]:
from table_loading import *

In [2]:
# get rid of ALMA points for comparison?
almaoverride = False

# select ALMA-IMF field of interest
fields_to_fit = None

In [8]:
# utility function to read object sizes in more convenient measurements
def bytesto(bytes, to, bsize=1024):
    """convert bytes to megabytes, etc.
       sample code:
           print('mb= ' + str(bytesto(314575262000000, 'm')))
       sample output: 
           mb= 300002347.946
    """

    a = {'k' : 1, 'm': 2, 'g' : 3, 't' : 4, 'p' : 5, 'e' : 6 }
    r = float(bytes)
    for i in range(a[to]):
        r = r / bsize

    return(r)

In [ ]:
if fields_to_fit == None:
    # Call 'None' to fit all fields in the sample (will take a while!)
    fields_to_fit = ['G10','G12','W43MM1','W43MM2','W43MM3','W51-E','W51IRS2',
                     'G008','G327','G328','G333','G337','G338','G351','G353']

fit_results_tbl = Table(data=np.zeros(0, dtype=[('SPICY', 'int64'), ('geometry', 'str'),
                                            ('source.flux', 'str'),('source.error', 'str'),
                                            ('source.valid', 'str'),('MODEL_NAME', 'str'),
                                            ('chi2', 'float64'),('av', 'float64'),('sc', 'float64')]))
fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_result_test', format='fits', overwrite=True)
fit_results_tbl = None

for fieldid in tqdm(fields_to_fit):
    print(fieldid)
    region_fitinfo = {}

    # load table from Table Prep notebook
    tbl = Table.read(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/Region_tables/Unfitted/{fieldid}', format='fits') # !filepath!
    print("Sources: ",len(tbl))

    if almaoverride:
        for x in ['ALMA-IMF_3mm_flux','ALMA-IMF_3mm_eflux','ALMA-IMF_1mm_flux','ALMA-IMF_1mm_eflux']:
            tbl[x] = 0
            tbl[x] = np.nan
            tbl[x] = np.ma.masked

    # define constant sizes, based on literature values
    apertures = [2.4, 2.4, 2.4, 2.4, 6, 10, 13.5, 23, 30, 41, 3, 3]*u.arcsec
    
    # determine filters and add aperture sizes based on whether we're using UKDISS or VIRAC data
    ukidss_fields = ['G10','G12','W43MM1','W43MM2','W43MM3','W51-E','W51IRS2']
    virac_fields = ['G008','G327','G328','G333','G337','G338','G351','G353']
    
    if any(x in virac_fields for x in tbl['ALMAIMF_FIELDID']):
        print("Grabbing VIRAC filters")
        sed_filters, wavelength_dict, filternames, zpts = get_filters("south")
        apertures = apertures.insert(0,[1.415, 1.415, 1.415, 1.415, 1.415]*u.arcsec)
        hemisphere = "south"
    elif any(x in ukidss_fields for x in tbl['ALMAIMF_FIELDID']):
        print("Grabbing UKIDSS filters")
        sed_filters, wavelength_dict, filternames, zpts = get_filters("north")
        apertures = apertures.insert(0,[2, 2, 2]*u.arcsec)
        hemisphere = "north"
    print("Filters:"+str(filternames))
    print("Apertures:"+str(apertures))

    # read distance from table (in kpc)
    regiondistance = np.nanmax(tbl['Distance'])
    print("Region literature distance (kpc): "+str(regiondistance))
    
    # infer allowable distance range (in kpc)
    distance_range=[regiondistance-0.2, regiondistance+0.2]
    print("Distance range (kpc):"+str(distance_range))
    
    # determine appropriate extinction, using 2*region distance as a lower bound
    av_range=[regiondistance*2,60]
    print("Extinction range (kpc): "+str(av_range))
    
    # make extinction law
    extinction = make_extinction()
    
    # run fitting on each source in region, store data to an array
    for rownum, row in enumerate(tbl):
        fits = full_source_fit(tbl, rownum, filternames, apertures, 
                               '/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/robitaille_models', 
                               extinction, distance_range*u.kpc, av_range) # !filepath!
        
        # append fit data to data array
        region_fitinfo[tbl['SPICY'][rownum]] = fits
        
        #clean variables for memory
        fits = None
    
    new_tbl = construct_fitinfo_tbl(region_fitinfo)
    prev_tbl = Table.read('/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_result_test')
    fit_results_tbl = vstack([prev_tbl, new_tbl])
    print(bytesto(np.sum([np.sum([sys.getsizeof(fit_results_tbl[colname][n]) for n in np.arange(len(fit_results_tbl))]) for colname in fit_results_tbl.keys()]),'g'))

    new_tbl = None
    prev_tbl = None
    region_fitinfo = None
    
    fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_result_test', format='fits', overwrite=True)
    fit_results_tbl = None

In [7]:
len(set(prev_tbl['SPICY']))

107

In [ ]:
print("SUCCESS")

In [7]:
print(bytesto(np.sum([np.sum([sys.getsizeof(fit_results_tbl[colname][n]) for n in np.arange(len(fit_results_tbl))]) for colname in fit_results_tbl.keys()]),'g'))

new_tbl = None
prev_tbl = None
region_fitinfo = None
    
fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_result_test', format='fits', overwrite=True)
fit_results_tbl = None

0.0002412833273410797


In [4]:
fit_results_tbl = vstack([prev_tbl, new_tbl])

In [23]:
fit_results_tbl

SPICY,geometry,source.flux,source.error,source.valid,MODEL_NAME,chi2,av,sc
,,,,,,,,
int64,str7,str252,str144,str43,str30,float64,float64,float64
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",InkaLcmQ_05,14.943552423272461,56.717855413543475,0.7481880270062005
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",qCK4JuEZ_02,16.663449494166358,56.372907721405376,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",qCK4JuEZ_03,16.674636010503864,53.41318306552333,0.7320956853204998
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",jPkU2W8J_09,16.692403203749862,49.45955329902702,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",jPkU2W8J_08,16.69544922691014,49.48522349673272,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",jPkU2W8J_07,16.698474286845112,49.70502730398237,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",qCK4JuEZ_04,16.727191111826183,51.07673910922111,0.7481880270062005
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969

In [24]:
fit_results_tbl.convert_unicode_to_bytestring()

In [25]:
fit_results_tbl

SPICY,geometry,source.flux,source.error,source.valid,MODEL_NAME,chi2,av,sc
,,,,,,,,
int64,bytes7,bytes252,bytes144,bytes43,bytes30,float64,float64,float64
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",InkaLcmQ_05,14.943552423272461,56.717855413543475,0.7481880270062005
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",qCK4JuEZ_02,16.663449494166358,56.372907721405376,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",qCK4JuEZ_03,16.674636010503864,53.41318306552333,0.7320956853204998
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",jPkU2W8J_09,16.692403203749862,49.45955329902702,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",jPkU2W8J_08,16.69544922691014,49.48522349673272,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",jPkU2W8J_07,16.698474286845112,49.70502730398237,0.7160033436347992
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.12093854969, 751532.1432824293, 607054.9121893466, 288241.04181283846, nan, nan","0.997, 0.997, 0.997, 0.1305228665653293, 1.585589995323331, 1.4611468439681023, 0.997, nan, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 1, 3, 0, 3, 3, 3, 3, 3, 0, 0",qCK4JuEZ_04,16.727191111826183,51.07673910922111,0.7481880270062005
102000,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.9489032300691156, 9.709511941433862, 12.741200714826622, 2.519363851788965, nan, 39001.312800087384, 56891.1

In [29]:
bytesto(np.sum([np.sum([sys.getsizeof(fit_results_tbl[colname][n]) for n in np.arange(len(fit_results_tbl))]) for colname in fit_results_tbl.keys()]),'g')

0.6474301367998123

In [12]:
[sys.getsizeof(fit_results_tbl[colname]) for colname in fit_results_tbl.keys()]

[136, 136, 136, 136, 136, 136, 136, 136, 136]

In [ ]:
t.convert_unicode_to_bytestring()

In [17]:
sys.getsizeof(all_fitinfo)

48

In [ ]:
vstack([fitinfo_tbl_by_region[fieldid] for fieldid in fitinfo_tbl_by_region.keys()])

In [12]:
fitinfo_tbl_by_region.values()

TypeError: 'dict_values' object is not subscriptable

In [11]:
[fitinfo_tbl_by_region[fieldid] for fieldid in fitinfo_tbl_by_region.keys()]

[<Table length=14967>
 SPICY geometry ...         av                 sc        
                ...                                      
 int64   str7   ...      float64            float64      
 ----- -------- ... ------------------ ------------------
 75717  spu-smi ... 59.659678674870776  0.711807229041191
 75725  s-pbhmi ...  37.95101820680336  0.711807229041191
 75725  s-pbhmi ... 37.190083266916545  0.711807229041191
 75725  s-pbhmi ... 30.719427171577554  0.711807229041191
 75725  s-pbsmi ...  28.01897313129891  0.711807229041191
 75725  s-pbsmi ...  38.74502259758186 0.6766936096248666
 75725  s-pbsmi ... 24.919223631893882  0.711807229041191
 75725  s-pbsmi ...  40.92140798939504 0.6766936096248666
 75725  s-pbsmi ...  11.98116579006977  0.711807229041191
   ...      ... ...                ...                ...
 75788  spubhmi ...  58.90255015434248 0.6766936096248666
 75788  spubhmi ...  20.07856763640905  0.711807229041191
 75788  spubhmi ...  59.71415958467156 0.676693609

In [13]:
vstack([fitinfo_tbl_by_region[fieldid] for fieldid in fitinfo_tbl_by_region.keys()])

SPICY,geometry,source.flux,source.error,source.valid,MODEL_NAME,chi2,av,sc
,,,,,,,,
int64,str7,str331,str241,str49,str30,float64,float64,float64
75717,spu-smi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 1.4811193944511212, 1.826407260695228, 2.3568491729598557, 7.075708493104115, 3770.6773681640625, 62736.55310072545, 55176.82815332446, 699890.6664243827, 471583.10983309546, 266739.6554715331, nan, nan","0.997, 0.997, 0.997, 0.18887668560507512, 0.2312331967922324, 0.997, 1.0581105659177457, 0.997, 0.997, 0.997, 0.997, 0.997, 0.997, nan, nan","3, 3, 3, 1, 1, 3, 1, 3, 3, 3, 3, 3, 3, 0, 0",E3FGhkZy_09,2.0993100434174035,59.659678674870776,0.711807229041191
75725,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 24.76223841668025, 41.84058686326525, 64.09294076550027, 67.32396745462171, 1302.0323181152344, 5637.371220491355, 13964.018572858193, 218493.11686264555, 188738.23779058584, 105059.29297132936, 3.3458597080412793, 0.15990745571439244","0.997, 0.997, 0.997, 1.2494707153290499, 1.804135265011802, 2.528437207397165, 2.7176645747001813, 0.997, 0.997, 0.997, 0.997, 0.997, 0.997, 0.11638487988885025, 0.997","3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3",InkaLcmQ_01,46.928228263196196,37.95101820680336,0.711807229041191
75725,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 24.76223841668025, 41.84058686326525, 64.09294076550027, 67.32396745462171, 1302.0323181152344, 5637.371220491355, 13964.018572858193, 218493.11686264555, 188738.23779058584, 105059.29297132936, 3.3458597080412793, 0.15990745571439244","0.997, 0.997, 0.997, 1.2494707153290499, 1.804135265011802, 2.528437207397165, 2.7176645747001813, 0.997, 0.997, 0.997, 0.997, 0.997, 0.997, 0.11638487988885025, 0.997","3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3",InkaLcmQ_02,48.07575642140455,37.190083266916545,0.711807229041191
75725,s-pbhmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 24.76223841668025, 41.84058686326525, 64.09294076550027, 67.32396745462171, 1302.0323181152344, 5637.371220491355, 13964.018572858193, 218493.11686264555, 188738.23779058584, 105059.29297132936, 3.3458597080412793, 0.15990745571439244","0.997, 0.997, 0.997, 1.2494707153290499, 1.804135265011802, 2.528437207397165, 2.7176645747001813, 0.997, 0.997, 0.997, 0.997, 0.997, 0.997, 0.11638487988885025, 0.997","3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3",InkaLcmQ_03,66.40320136257417,30.719427171577554,0.711807229041191
75725,s-pbsmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 24.76223841668025, 41.84058686326525, 64.09294076550027, 67.32396745462171, 1302.0323181152344, 5637.371220491355, 13964.018572858193, 218493.11686264555, 188738.23779058584, 105059.29297132936, 3.3458597080412793, 0.15990745571439244","0.997, 0.997, 0.997, 1.2494707153290499, 1.804135265011802, 2.528437207397165, 2.7176645747001813, 0.997, 0.997, 0.997, 0.997, 0.997, 0.997, 0.11638487988885025, 0.997","3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3",5ZvkZvTO_01,51.447651771942056,28.01897313129891,0.711807229041191
75725,s-pbsmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 24.76223841668025, 41.84058686326525, 64.09294076550027, 67.32396745462171, 1302.0323181152344, 5637.371220491355, 13964.018572858193, 218493.11686264555, 188738.23779058584, 105059.29297132936, 3.3458597080412793, 0.15990745571439244","0.997, 0.997, 0.997, 1.2494707153290499, 1.804135265011802, 2.528437207397165, 2.7176645747001813, 0.997, 0.997, 0.997, 0.997, 0.997, 0.997, 0.11638487988885025, 0.997","3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 1, 3",YBciyFpv_03,55.836664579527465,38.74502259758186,0.6766936096248666
75725,s-pbsmi,"0.05048449145906865, 0.07708000631399106, 0.057309295140499344, 24.76223841668025, 41.84058686326525, 64.09294076550027, 67.32396745462171, 1302.0323181152344, 5637.371220491355, 13964.018572858193, 218493.11686264555, 188738.23779058584, 105059.29297132936, 3.3458597080412793, 0.15990745571439244","0.997, 0.

In [33]:
bytesto(np.sum([np.sum([sys.getsizeof(all_fitinfo[spicyid][geom]) for geom in geometries]) for spicyid in all_fitinfo.keys()]),'k')

6.75

In [5]:
print(bytesto(sys.getsizeof(np.sum([np.sum([sys.getsizeof(all_fitinfo[spicyid][geom]) for geom in geometries]) for spicyid in all_fitinfo.keys()])),'m')," MB / ",bytesto(sys.getsizeof(np.sum([np.sum([sys.getsizeof(all_fitinfo[spicyid][geom]) for geom in geometries]) for spicyid in all_fitinfo.keys()])),'k')," KB / ",bytesto(sys.getsizeof(np.sum([np.sum([sys.getsizeof(all_fitinfo[spicyid][geom]) for geom in geometries]) for spicyid in all_fitinfo.keys()])),'g')," GB")

3.0517578125e-05  MB /  0.03125  KB /  2.9802322387695312e-08  GB


In [16]:
np.sum([np.sum([sys.getsizeof(all_fitinfo[spicyid][geom]) for geom in geometries]) for spicyid in all_fitinfo.keys()])

4320

In [12]:
all_fitinfo[75717]

{'s-pbhmi': <sedfitter.fit_info.FitInfo at 0x14ae1a6f7eb0>,
 's-pbsmi': <sedfitter.fit_info.FitInfo at 0x14ae520a1910>,
 'sp--h-i': <sedfitter.fit_info.FitInfo at 0x14ae1a767be0>,
 's-p-hmi': <sedfitter.fit_info.FitInfo at 0x14ae1a7611c0>,
 'sp--hmi': <sedfitter.fit_info.FitInfo at 0x14ae1a6fbeb0>,
 'sp--s-i': <sedfitter.fit_info.FitInfo at 0x14ae1a703700>,
 's-p-smi': <sedfitter.fit_info.FitInfo at 0x14ae1a797220>,
 'sp--smi': <sedfitter.fit_info.FitInfo at 0x14ae1a797b20>,
 'spubhmi': <sedfitter.fit_info.FitInfo at 0x14ae1a6fb130>,
 'spubsmi': <sedfitter.fit_info.FitInfo at 0x14ae1a76da90>,
 'spu-hmi': <sedfitter.fit_info.FitInfo at 0x14ae1a758be0>,
 'spu-smi': <sedfitter.fit_info.FitInfo at 0x14ae1a797d90>,
 's---s-i': <sedfitter.fit_info.FitInfo at 0x14ae1a761700>,
 's---smi': <sedfitter.fit_info.FitInfo at 0x14ae1a85f760>,
 's-ubhmi': <sedfitter.fit_info.FitInfo at 0x14ae1a740fd0>,
 's-ubsmi': <sedfitter.fit_info.FitInfo at 0x14ae1b6d7b50>,
 's-u-hmi': <sedfitter.fit_info.FitInfo 

In [34]:
fitinfo_tbl = construct_fitinfo_tbl(all_fitinfo)

  0%|          | 0/8 [00:00<?, ?it/s]

75717


  0%|          | 0/18 [00:00<?, ?it/s]

75725


  0%|          | 0/18 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

75767


  0%|          | 0/18 [00:00<?, ?it/s]

75788


  0%|          | 0/18 [00:00<?, ?it/s]

In [35]:
sys.getsizeof(fitinfo_tbl)

48

In [30]:
fit_results_tbl.write(f'/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_result_test', format='fits', overwrite=True)

In [19]:
sys.getsizeof('/blue/adamginsburg/adamginsburg/SPICY_ALMAIMF/BriceTingle/fit_result_test')

122